In [ ]:
!pip install torch transformers datasets pandas numpy sqlalchemy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
import pandas as pd
import re
import sqlparse

# Load Spider dataset
dataset = load_dataset("spider")

# Convert to Pandas DataFrames
train_df = pd.DataFrame(dataset["train"])
validation_df = pd.DataFrame(dataset["validation"])

# Function to clean natural language queries
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r"[^a-zA-Z0-9_*\s]", "", text)  # Keep *, letters, numbers, and spaces
    return text

# Function to normalize SQL queries (fix formatting)
def normalize_sql(query):
    query = sqlparse.format(query, reindent=True, keyword_case="lower")  # Format SQL properly
    return query.strip()

# Apply preprocessing
train_df["question"] = train_df["question"].apply(clean_text)
train_df["query"] = train_df["query"].apply(normalize_sql)

validation_df["question"] = validation_df["question"].apply(clean_text)
validation_df["query"] = validation_df["query"].apply(normalize_sql)

# Save cleaned data
train_df.to_csv("spider_train_cleaned.csv", index=False)
validation_df.to_csv("spider_validation_cleaned.csv", index=False)

print("✅ Preprocessing complete! Cleaned data saved.")


✅ Preprocessing complete! Cleaned data saved.


In [ ]:
import torch
from transformers import AutoTokenizer

# Load tokenizer (T5 Large)
tokenizer = AutoTokenizer.from_pretrained("t5-large")

# Function to preprocess data for training
def preprocess_data(examples):
    inputs = ["translate to SQL: " + q for q in examples["question"]]
    targets = examples["query"]

    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
train_data = dataset["train"].map(preprocess_data, batched=True, remove_columns=["question_toks", "query_toks", "query_toks_no_value"])
val_data = dataset["validation"].map(preprocess_data, batched=True, remove_columns=["question_toks", "query_toks", "query_toks_no_value"])

# Save tokenized tensors
torch.save(train_data, "train_data.pt")
torch.save(val_data, "val_data.pt")

print("✅ Tokenization complete! Data saved.")


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

✅ Tokenization complete! Data saved.


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load T5 model (use "t5-3b" for even better performance)
model = T5ForConditionalGeneration.from_pretrained("t5-large")

# Training arguments
training_args = TrainingArguments(
    output_dir="./t5_sql_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Increased from 5 to 10 for better learning
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True if torch.cuda.is_available() else False,  # Use FP16 if GPU available
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

# Train the model
trainer.train()

# Save model
model.save_pretrained("t5_sql_model")
tokenizer.save_pretrained("t5_sql_model")

print("✅ Training Complete! Model saved.")



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 22ceuon127 (22ceuon127-navneet-trading-company) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
